In [32]:
import numpy as np
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt
import numpy as np
import torch.nn.functional as F
import math
import copy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, matthews_corrcoef, f1_score
import random

In [33]:
torch.set_default_tensor_type(torch.DoubleTensor)

In [34]:
pos_table = pd.read_csv('/Users/jiaming/Desktop/revision/datas/pos_encoding_OH_ND.csv')
neg_table = pd.read_csv('/Users/jiaming/Desktop/revision/datas/neg_encoding_OH_ND.csv')

pos_table = pos_table.iloc[:,1:]
neg_table = neg_table.iloc[:,1:]

In [35]:
pos_geo = pd.read_csv('/Users/jiaming/Desktop/revision/datas/pos_domain_encoding.csv')
neg_geo = pd.read_csv('/Users/jiaming/Desktop/revision/datas/neg_domain_encoding.csv')

pos_geo = pos_geo.iloc[:,1:]
neg_geo = neg_geo.iloc[:,1:]

In [36]:
pos_cb = pd.concat([pos_table, pos_geo], axis=1)
neg_cb = pd.concat([neg_table, neg_geo], axis=1)[:1891]

pos_np = pos_cb.to_numpy()
neg_np = neg_cb.to_numpy()

raw_datas = np.concatenate((pos_np, neg_np), axis=0)
raw_labels = np.concatenate(([1] * pos_np.shape[0], [0] * neg_np.shape[0]), axis=0)

In [37]:
data_train, data_test, labels_train, labels_test = train_test_split(raw_datas, raw_labels, test_size=0.2, random_state=12)

In [38]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [39]:
N_seq = 49 # 245/41 41*5+40*1
def dataset_CNN(Xdataset, ydataset):
    Xydataset = []
    for iS in range(Xdataset.shape[0]):
        Xdataset_i = Xdataset[iS]
        ydataset_i = ydataset[iS]
        Xydataset_i = (torch.tensor(np.transpose(Xdataset_i.reshape(N_seq, 5)).reshape(1, N_seq, 5)), ydataset_i)
        Xydataset.append(Xydataset_i)
        
    return Xydataset

In [40]:
def actfunc(activation='relu'):
    if activation == 'relu':
        act = nn.ReLU
    return act()

In [41]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        # self.norm = LayerNorm(layer.size)

    def forward(self, x):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x)
        return x

In [42]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, len=None, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.len = len
        self.dropout = nn.Dropout(p=dropout)
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        # pe = torch.zeros(max_len, 1)
        # x=torch.cat([x,self.pe[:x.size(-2)]],dim=-2)
        position = torch.arange(0, max_len).unsqueeze(1)
        position *= 2
        div_term = torch.exp(position / d_model * math.log(10000))
        pe[:, 0::2] = torch.sin(position / div_term)
        pe[:, 1::2] = torch.cos(position / div_term)
        # pe = pe.unsqueeze(0)
        pe.requires_grad = False
        pe = pe / 10
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(-2)]
        return x

In [43]:
class SelfAttention(nn.Module):
    def __init__(self, inputsize, headnum=8, modelsize=None):
        super(SelfAttention, self).__init__()
        if modelsize is None:
            modelsize = inputsize // headnum
        self.Wq = clones(nn.Linear(inputsize, modelsize, bias=False), headnum)
        self.Wk = clones(nn.Linear(inputsize, modelsize), headnum)
        self.Wv = clones(nn.Linear(inputsize, modelsize), headnum)
        self.size = 1 / (modelsize ** 0.5)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, m=None):
        z = []
        if m is None:
            m = x
        for i in range(len(self.Wq)):
            q = self.Wq[i](x)
            k = self.Wk[i](m).transpose(-1, -2)
            weight = torch.mul(torch.matmul(q, k), self.size)
            v = torch.matmul(self.softmax(weight), self.Wv[i](m))
            z.append(v)
        z = torch.cat(z, -1)
        return z

In [44]:
class Conv1dtranspose(nn.Module):
    def __init__(self, in_chan, out_chan, kernel_size=3, stride=1, dilation=1, padding=0, pooling=False,
                 in_transpose=False, out_transpose=False, groups=1, dropout=0.1,acti='relu'):
        super(Conv1dtranspose, self).__init__()
        if padding == 'same':
            padding = kernel_size // 2
        self.conv = nn.Conv1d(in_channels=in_chan, out_channels=out_chan, padding=padding, groups=groups,
                              kernel_size=kernel_size, stride=stride, dilation=dilation)
        self.in_transpose = in_transpose
        self.out_transpose = out_transpose
        self.dropout = nn.Dropout(dropout)
        self.out=actfunc(acti)
        self.pooling = pooling
        if pooling:
            self.pool = nn.MaxPool1d(2)

    def forward(self, x, in_transpose=False):
        if in_transpose:
            x = torch.transpose(x, -1, -2)
        elif self.in_transpose:
            x = torch.transpose(x, -1, -2)
            x = self.conv(x)
            x = self.out(self.dropout(x))
        if self.pooling:
            x = self.pool(x)
        if self.out_transpose:
            x = torch.transpose(x, -1, -2)
        return x

In [45]:
class LayerNorm(nn.Module): 
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features),requires_grad=True)
        self.b_2 = nn.Parameter(torch.zeros(features),requires_grad=True)
        self.eps = eps

    def forward(self, x):
        # print(x.size())
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


In [46]:
class Encoderlayer(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, inputsize, outsize, dropout=0.2, modelsize=None,acti='relu', headnum=16, fourier=False, res=False):
        super(Encoderlayer, self).__init__()
        if modelsize is None:
            self.res = True
            modelsize = int(inputsize / headnum)
        else:
            self.res = False
        if modelsize * headnum == outsize:
            self.resout = True
        else:
            self.resout = False
        if fourier:
            self.att = rfft2()
        else:
            self.att = SelfAttention(inputsize, headnum, modelsize)
        self.Wz = nn.Sequential(nn.Linear(modelsize * headnum, outsize), actfunc(acti), nn.Linear(outsize, outsize))
        self.sublayer = [LayerNorm(inputsize), LayerNorm(modelsize * headnum)]
        self.dropout = clones(nn.Dropout(dropout), 3)

    def forward(self, x):
        if self.res:
            z = x + self.dropout[0](self.att(self.sublayer[0](x)))
        else:
            z = self.att(x)
        if self.resout:
            out = z + self.dropout[1](self.Wz(self.sublayer[1](z)))
        else:
            out = self.Wz(z)
        return out

In [47]:
class rfft2(nn.Module):
    def __init__(self):
        super(rfft2, self).__init__()

    def forward(self, x):
        return torch.fft.rfft2(x)

In [48]:
class CLS(nn.Module):
    def __init__(self, in_size, out_size=100,acti='relu'):
        super(CLS, self).__init__()
        self.model = nn.Sequential(nn.Linear(in_size, 1000),
                                   actfunc(acti),
                                   nn.Linear(1000, out_size), nn.Sigmoid())

    def forward(self, x):
        x = x[0]
        out = self.model(x).view([1,-1])
        return out

In [49]:
class Transformer(nn.Module):
    def __init__(self, N=5, src_vocab=64,d_model=256, dropout=0.25, h=8,outsize=2,acti='relu'):
        super(Transformer, self).__init__()
        self.Embed = nn.Sequential(
            Conv1dtranspose(src_vocab, d_model,acti=acti, in_transpose=True, out_transpose=True, kernel_size=7, stride=5),
            nn.Flatten(0, 1))
        self.model = nn.Sequential(
            PositionalEncoding(d_model, dropout),
            Encoder(Encoderlayer(inputsize=d_model, outsize=d_model, headnum=h, dropout=dropout,acti=acti), N),
            CLS(d_model,out_size=outsize,acti=acti)
        )
        self.cls = (torch.ones([1, d_model]) * -1)

    def forward(self, x):
        x = self.Embed(x)
        x = torch.cat([self.cls, x], dim=0)
        x = self.model(x)
        return x

In [53]:
model = Transformer(src_vocab = 5) 
criterion = nn.CrossEntropyLoss()
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#N_test=len(train_dataset)
n_epochs = 2

In [62]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for epoch in range(n_epochs):
    print("epoch: ", epoch)
    
    i=1
    for train_dataset_indx, val_dataset_indx in kf.split(X=data_train):
        print("this is spilt:", i)
        i=i+1
        
        data_train_splited = data_train[train_dataset_indx]
        data_val_splited = data_train[val_dataset_indx]
        labels_train_splited = labels_train[train_dataset_indx]
        labels_val_splited = labels_train[val_dataset_indx]
        
        train_dataset = dataset_CNN(data_train_splited, labels_train_splited)
        val_dataset = dataset_CNN(data_val_splited, labels_val_splited)
        
        
        for x, y in train_dataset:
        
            optimizer.zero_grad()
            z = model(x)
            y = torch.tensor(np.array(y).reshape(1,))
            y = y.to(torch.long)
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            
        with torch.set_grad_enabled(False):
            N_test=len(val_dataset)
            prediction = []
            for x_1, y_1 in val_dataset:
                model.eval()
                z = model(x_1)
                pred = z[:, 1]
                for z_i in pred:
                    prediction.append(z_i.item())
                        
            prediction = np.array(prediction)
            y_hat_test = np.round(prediction)
            y_label = labels_val_splited
                
            TP = np.sum(y_label[y_label == y_hat_test] == 1)
            TN = np.sum(y_label[y_label == y_hat_test] == 0)
            FP = np.sum(y_hat_test[y_label != y_hat_test] == 1)
            FN = np.sum(y_hat_test[y_label != y_hat_test] == 0)
            Pre = TP/(TP+FP)
            Sn = TP/(TP+FN)
            Sp = TN/(TN+FP)
            MCC = matthews_corrcoef(y_label, y_hat_test)
            AUC = roc_auc_score(y_label, prediction)
            ACC = (TP+TN)/(TP+TN+FP+FN)
            F1= f1_score(y_label, y_hat_test)
            
            print(Sn, Sp, ACC, F1, MCC, AUC)

epoch:  0
this is spilt: 1
0.9228187919463087 0.8051948051948052 0.863036303630363 0.8688783570300157 0.7320081763683246 0.9391070339056916
this is spilt: 2
0.9704918032786886 0.6966666666666667 0.8347107438016529 0.8554913294797687 0.694793619803805 0.9352677595628415
this is spilt: 3
0.9404388714733543 0.7797202797202797 0.8644628099173554 0.8797653958944281 0.7339151390076786 0.9383015104018239
this is spilt: 4
0.9047619047619048 0.8713826366559485 0.8876033057851239 0.8866666666666667 0.7758900093736828 0.9450423256119168
this is spilt: 5
0.8566775244299675 0.8926174496644296 0.8743801652892562 0.8737541528239203 0.7494424424645333 0.9323830968672802
epoch:  1
this is spilt: 1
0.9530201342281879 0.6915584415584416 0.8201320132013201 0.8389955686853766 0.6657739881676709 0.950013074174148
this is spilt: 2
0.9606557377049181 0.8866666666666667 0.9239669421487603 0.9272151898734178 0.8500861933371497 0.9537486338797815
this is spilt: 3
0.9373040752351097 0.8776223776223776 0.909090909

In [63]:
test_dataset = dataset_CNN(data_test, labels_test)
N_test=len(test_dataset)
prediction = []
for x_2, y_2 in test_dataset:
    model.eval()
    z = model(x_2)
    pred = z[:, 1]
    for z_i in pred:
        prediction.append(z_i.item())

prediction = np.array(prediction)
y_hat_test = np.round(prediction)
y_label = labels_test

In [67]:
np.save("./seq_plus_geo_prediction.npy", prediction)
np.save("./seq_plus_geo_y_label.npy", y_label)

In [64]:
TP = np.sum(y_label[y_label == y_hat_test] == 1)
TN = np.sum(y_label[y_label == y_hat_test] == 0)
FP = np.sum(y_hat_test[y_label != y_hat_test] == 1)
FN = np.sum(y_hat_test[y_label != y_hat_test] == 0)
Pre = TP/(TP+FP)
Sn = TP/(TP+FN)
Sp = TN/(TN+FP)
MCC = matthews_corrcoef(y_label, y_hat_test)
AUC = roc_auc_score(y_label, prediction)
ACC = (TP+TN)/(TP+TN+FP+FN)
F1= f1_score(y_label, y_hat_test)

In [65]:
print(Sn, Sp, ACC, F1, MCC, AUC)

0.8075880758807588 0.8324742268041238 0.8203434610303831 0.814207650273224 0.640397860099797 0.8678233174084317
